# Digital Twin from Image using MiDaS, MediaPipe, Open3D, and Plotly

This notebook processes a single image (or a frame extracted from a video) to create a 3D digital twin. It performs the following steps:

1. **Depth Estimation** using MiDaS to generate a relative depth map.
2. **Pose Estimation** using MediaPipe to extract 2D skeleton landmarks.
3. **Point Cloud Generation** using Open3D from the RGB-D data.
4. **Visualization** using Plotly to show (a) the raw point cloud and (b) the digital twin (point cloud with overlaid skeleton).

Upload your test image (e.g. `input.jpg`) to the Colab file system and run the cells in order.

In [ ]:
!apt-get update
!apt-get install -y libosmesa6-dev libgl1-mesa-glx libglfw3

!pip install opencv-python-headless mediapipe open3d torch torchvision plotly

In [ ]:
import cv2
import torch
import numpy as np
import mediapipe as mp
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from torchvision import transforms
from IPython.display import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Enable cuDNN benchmarking for optimal performance
torch.backends.cudnn.benchmark = True

## Load MiDaS Model

We use the small MiDaS model for faster inference.

In [ ]:
model_type = "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.small_transform if model_type == "MiDaS_small" else midas_transforms.default_transform
print("MiDaS model loaded.")

## Setup MediaPipe Pose

We configure MediaPipe in static image mode for single-image processing.

In [ ]:
mp_pose = mp.solutions.pose
pose_estimator = mp_pose.Pose(static_image_mode=True,
                              model_complexity=1,
                              min_detection_confidence=0.5,
                              min_tracking_confidence=0.5)
pose_connections = mp_pose.POSE_CONNECTIONS
print("MediaPipe Pose loaded.")

## Load Input Image

Upload your test image (e.g., `input.jpg`) to the Colab file system and update the image path below.

In [ ]:
# Update 'input.jpg' with the path to your test image
image_path = "input.jpg"
frame = cv2.imread(image_path)
if frame is None:
    raise ValueError("Could not load the image. Please check the path.")
print("Image loaded successfully.")

## Depth Estimation with MiDaS

Optionally, downscale the image for faster inference.

In [ ]:
downscale_factor = 1.0  # Change to 0.5 (or another value) to downscale the image
orig_height, orig_width, _ = frame.shape
proc_width = int(orig_width * downscale_factor)
proc_height = int(orig_height * downscale_factor)
frame_proc = cv2.resize(frame, (proc_width, proc_height), interpolation=cv2.INTER_AREA)
frame_rgb = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2RGB)

with torch.no_grad():
    input_batch = transform(frame_rgb).to(device)
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=(proc_height, proc_width),
        mode="bilinear",
        align_corners=False
    ).squeeze()
torch.cuda.synchronize()
depth_map = prediction.cpu().detach().numpy()
depth_map_norm = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX)
print("Depth estimation complete.")

## Create 3D Point Cloud using Open3D

We create an RGB-D image from the color image and the estimated depth map, then generate a point cloud.

In [ ]:
o3d_color = o3d.geometry.Image(frame_rgb)
o3d_depth = o3d.geometry.Image((depth_map_norm * 1000).astype(np.uint16))
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d_color, o3d_depth,
    depth_scale=1000.0,
    convert_rgb_to_intensity=False
)

# Approximate camera intrinsics based on the processed image size
fx = fy = proc_width  # Simplistic assumption
ppx = proc_width / 2
ppy = proc_height / 2
intrinsic = o3d.camera.PinholeCameraIntrinsic(proc_width, proc_height, fx, fy, ppx, ppy)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)

# Adjust point cloud orientation
pcd.transform([[1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, 1]])
print("3D point cloud created.")

## Pose Estimation and 3D Skeleton Creation

We use MediaPipe to extract 2D pose landmarks from the image, then back-project them to 3D using the depth map. Adjust the maximum depth scaling as needed.

In [ ]:
def backproject_point(u, v, depth_value, fx, fy, ppx, ppy):
    z = depth_value
    x = (u - ppx) * z / fx
    y = (v - ppy) * z / fy
    return np.array([x, y, z])

max_depth_meters = 5.0  # Adjust if necessary
results = pose_estimator.process(frame_rgb)
keypoints_3d = []
if results.pose_landmarks:
    for landmark in results.pose_landmarks.landmark:
        u = int(landmark.x * proc_width)
        v = int(landmark.y * proc_height)
        u_clamped = np.clip(u, 0, proc_width - 1)
        v_clamped = np.clip(v, 0, proc_height - 1)
        depth_val = depth_map_norm[v_clamped, u_clamped]
        depth_in_meters = depth_val * max_depth_meters
        keypoints_3d.append(backproject_point(u, v, depth_in_meters, fx, fy, ppx, ppy))
else:
    keypoints_3d = [np.array([0, 0, 0]) for _ in range(33)]
keypoints_3d = np.array(keypoints_3d)
print("3D skeleton (pose landmarks) created.")

## Visualize Point Cloud using Plotly

This cell displays an interactive 3D scatter plot of the point cloud.

In [ ]:
# Convert Open3D point cloud to NumPy arrays
pts = np.asarray(pcd.points)
if len(pcd.colors) > 0:
    colors = np.asarray(pcd.colors)
else:
    colors = np.ones((pts.shape[0], 3))

# Create a Plotly 3D scatter trace for the point cloud
pcd_trace = go.Scatter3d(
    x=pts[:, 0],
    y=pts[:, 1],
    z=pts[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color=['rgb({},{},{})'.format(int(c[0]*255), int(c[1]*255), int(c[2]*255)) for c in colors],
        opacity=0.8
    ),
    name='Point Cloud'
)

fig_pointcloud = go.Figure(data=[pcd_trace])
fig_pointcloud.update_layout(scene=dict(aspectmode='data'),
                             title="Point Cloud Only")
fig_pointcloud.show()

## Visualize Digital Twin (Point Cloud + Skeleton) using Plotly

This cell overlays the 3D skeleton (pose landmarks) onto the point cloud.

In [ ]:
# Create line traces for the skeleton
line_traces = []
for connection in pose_connections:
    start_idx, end_idx = connection
    if start_idx < len(keypoints_3d) and end_idx < len(keypoints_3d):
        p0 = keypoints_3d[start_idx]
        p1 = keypoints_3d[end_idx]
        line_trace = go.Scatter3d(
            x=[p0[0], p1[0]],
            y=[p0[1], p1[1]],
            z=[p0[2], p1[2]],
            mode='lines',
            line=dict(color='green', width=5),
            showlegend=False
        )
        line_traces.append(line_trace)

fig_digital_twin = go.Figure(data=[pcd_trace] + line_traces)
fig_digital_twin.update_layout(scene=dict(aspectmode='data'),
                              title="Digital Twin - 3D Model with Skeleton")
fig_digital_twin.show()